## VacationPy

### Obervations





In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Export csv
cities_df = pd.read_csv("../output_data/city_data.csv")

# drop NA values
# cities_df = cities_df.dropna()
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapoeta,4.7717,33.5903,85.68,45,100,11.83,SS,05-19-2021
1,Pevek,69.7008,170.3133,21.09,89,39,8.61,RU,05-19-2021
2,Bontang,0.1333,117.5000,78.01,90,100,7.43,ID,05-19-2021
3,Hilo,19.7297,-155.0900,70.79,86,90,10.36,US,05-19-2021
4,Tomatlán,19.9333,-105.2500,73.92,65,0,2.44,MX,05-19-2021
...,...,...,...,...,...,...,...,...,...
613,Zunyi,27.6867,106.9072,63.03,95,100,3.65,CN,05-19-2021
614,Natal,-5.7950,-35.2094,88.02,69,40,10.56,BR,05-19-2021
615,Inírida,3.8653,-67.9239,72.12,99,100,1.92,CO,05-19-2021
616,Lovozero,68.0050,35.0117,69.19,54,100,2.59,RU,05-19-2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Plot Heatmap

# Set up variables
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)
fig = gmaps.figure(center=(16.522731, -5.042275), zoom_level=2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values

In [5]:
#reduce cities to ideal conditions 
preferred_cities_df = cities_df.loc[(cities_df["Wind Speed"]<= 10) & (cities_df["Cloudiness"] == 0) & 
                                (cities_df["Max Temp"]>= 70) & (cities_df["Max Temp"]<= 80)].dropna()

# (cities_df["Humidity"]<= 80) &
preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Tomatlán,19.9333,-105.2500,73.92,65,0,2.44,MX,05-19-2021
130,Port Elizabeth,-33.9180,25.5701,74.10,53,0,5.75,ZA,05-19-2021
213,Cabo San Lucas,22.8909,-109.9124,72.81,72,0,9.24,MX,05-19-2021
300,Cap Malheureux,-19.9842,57.6142,77.47,65,0,5.01,MU,05-19-2021
389,Karratha,-20.7377,116.8463,75.42,65,0,3.94,AU,05-19-2021
406,Ortaköy,38.7373,34.0387,75.45,23,0,6.04,TR,05-19-2021
419,Grand Gaube,-20.0064,57.6608,77.38,65,0,5.01,MU,05-19-2021
502,Agua Verde,22.9000,-106.0500,78.13,74,0,7.02,MX,05-19-2021
562,Grand Baie,-20.0182,57.5802,77.45,65,0,5.01,MU,05-19-2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = preferred_cities_df[["City","Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,Tomatlán,MX,19.9333,-105.2500,
130,Port Elizabeth,ZA,-33.9180,25.5701,
213,Cabo San Lucas,MX,22.8909,-109.9124,
300,Cap Malheureux,MU,-19.9842,57.6142,
389,Karratha,AU,-20.7377,116.8463,
406,Ortaköy,TR,38.7373,34.0387,
419,Grand Gaube,MU,-20.0064,57.6608,
502,Agua Verde,MX,22.9000,-106.0500,
562,Grand Baie,MU,-20.0182,57.5802,


In [7]:
# Set up search criteria

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [8]:
# Iterate through dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 4: Tomatlán.
Closest hotel in Tomatlán is Norma franco.
------------
Retrieving Results for Index 130: Port Elizabeth.
Closest hotel in Port Elizabeth is 39 On Nile Guest House.
------------
Retrieving Results for Index 213: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 300: Cap Malheureux.
Closest hotel in Cap Malheureux is Pereybere Hotel & Spa.
------------
Retrieving Results for Index 389: Karratha.
Closest hotel in Karratha is ibis Styles Karratha.
------------
Retrieving Results for Index 406: Ortaköy.
Closest hotel in Ortaköy is Berilhan Hotel.
------------
Retrieving Results for Index 419: Grand Gaube.
Closest hotel in Grand Gaube is Veranda Paul et Virginie Hotel & Spa.
------------
Retrieving Results for Index 502: Agua Verde.
Missing field/result... skipping.
------------
Retrieving Results for Index 562: Grand Baie.
Closest hotel in Grand Baie is Trou aux Biches Beachcomber G

In [9]:
# Print the first hotel info
first_hotel = hotel_df[["City","Country", "Hotel Name"]].copy()
first_hotel.head(1)

,City,Country,Hotel Name
4,Tomatlán,MX,Norma franco


In [13]:
# Print updated hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,Tomatlán,MX,19.9333,-105.2500,Norma franco
130,Port Elizabeth,ZA,-33.9180,25.5701,39 On Nile Guest House
213,Cabo San Lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
300,Cap Malheureux,MU,-19.9842,57.6142,Pereybere Hotel & Spa
389,Karratha,AU,-20.7377,116.8463,ibis Styles Karratha
406,Ortaköy,TR,38.7373,34.0387,Berilhan Hotel
419,Grand Gaube,MU,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
502,Agua Verde,MX,22.9000,-106.0500,
562,Grand Baie,MU,-20.0182,57.5802,Trou aux Biches Beachcomber Golf Resort & Spa


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer on top of the heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))